In [1]:
!pip install praw
import requests
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import praw

In [2]:
'''
# Define subreddits and keywords
subreddits = ["Xiaomi", "smartphones", "Android"]
keywords = ["camera", "battery", "design", "display", "performance", "durability"]
model_keywords = [
    "14T Pro", 
    "14T-Pro", 
    "14TPro",
]
'''

In [3]:
'''
# Calculate timestamps for the last 6 months
end_date = int(datetime.utcnow().timestamp())  # Current timestamp
start_date = int((datetime.utcnow() - timedelta(days=180)).timestamp())  # 6 months ago
'''

In [4]:
'''
# Function to get posts from Pushshift API
def fetch_pushshift_data(subreddit, keywords, start_date, end_date, limit=100):
    base_url = "https://api.pushshift.io/reddit/search/submission/"
    all_posts = []

    for keyword in keywords:
        for model in model_keywords:
            search_query = f"{model} {keyword}"  # 组合机型和特性关键词
            params = {
                "subreddit": subreddit,
                "q": search_query,
                "after": start_date,
                "before": end_date,
                "size": limit,
                "sort": "desc"
            }
            
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json().get("data", [])
            for post in data:
                all_posts.append({
                    "subreddit": subreddit,
                    "title": post.get("title", ""),
                    "selftext": post.get("selftext", ""),
                    "created_utc": datetime.utcfromtimestamp(post["created_utc"]).strftime('%Y-%m-%d %H:%M:%S'),
                    "score": post.get("score", 0),
                    "num_comments": post.get("num_comments", 0),
                    "url": post.get("full_link", "")
                })
        else:
            print(f"Failed to fetch data for subreddit: {subreddit}, keyword: {keyword}")
    
    return all_posts

# Fetch data from all subreddits
all_posts = []
for subreddit in subreddits:
    print(f"Fetching data from subreddit: {subreddit}")
    posts = fetch_pushshift_data(subreddit, keywords, start_date, end_date, limit=100)
    all_posts.extend(posts)

# Convert to DataFrame
df = pd.DataFrame(all_posts)
'''

Fetching data from subreddit: Xiaomi
Failed to fetch data for subreddit: Xiaomi, keyword: camera
Failed to fetch data for subreddit: Xiaomi, keyword: battery
Failed to fetch data for subreddit: Xiaomi, keyword: design
Failed to fetch data for subreddit: Xiaomi, keyword: display
Failed to fetch data for subreddit: Xiaomi, keyword: performance
Failed to fetch data for subreddit: Xiaomi, keyword: durability
Fetching data from subreddit: smartphones
Failed to fetch data for subreddit: smartphones, keyword: camera
Failed to fetch data for subreddit: smartphones, keyword: battery
Failed to fetch data for subreddit: smartphones, keyword: design
Failed to fetch data for subreddit: smartphones, keyword: display
Failed to fetch data for subreddit: smartphones, keyword: performance
Failed to fetch data for subreddit: smartphones, keyword: durability
Fetching data from subreddit: Android
Failed to fetch data for subreddit: Android, keyword: camera
Failed to fetch data for subreddit: Android, keywo

In [2]:
import os

# 检查文件是否存在
print("Current working directory:", os.getcwd())
print("Files in directory:", os.listdir())
print(".env exists:", os.path.exists('.env'))

# 如果文件存在，读取其内容
if os.path.exists('.env'):
    with open('.env', 'r') as f:
        print("Contents of .env:")
        print(f.read())

Current working directory: C:\Users\Kevin\Text Analytics\Group Project
Files in directory: ['.env', '.ipynb_checkpoints', 'Scraping-data-from-amazon.ipynb', 'Scraping-data-from-reddit.ipynb', 'xiaomi_14Tpro_reddit_posts-180d.csv', 'xiaomi_14Tpro_reddit_posts-360d.csv', 'xiaomi_14Ultra_reddit_combined-180d.csv', 'xiaomi_14Ultra_reddit_posts-180d.csv', 'xiaomi_14Ultra_reddit_posts-360d.csv']
.env exists: True
Contents of .env:



In [3]:
from dotenv import load_dotenv
import os

# 重新加载环境变量
load_dotenv(override=True)

# 打印环境变量的值
print("Checking environment variables:")
print("CLIENT_ID:", os.getenv("CLIENT_ID"))
print("CLIENT_SECRET:", os.getenv("CLIENT_SECRET"))
print("USER_AGENT:", os.getenv("USER_AGENT"))
print("USERNAME:", os.getenv("USERNAME"))
print("PASSWORD:", os.getenv("PASSWORD"))

Checking environment variables:
CLIENT_ID: None
CLIENT_SECRET: None
USER_AGENT: None
USERNAME: Kevin
PASSWORD: None


In [4]:
# Load .env file
load_dotenv()

# Get credentials from environment variables
reddit = praw.Reddit(
    client_id="UcZ67U_Eg11j5HCa9norSw",
    client_secret="g0v7f2uuOgx6Ih9usmtcEIKe48Ww0A",
    user_agent="my_bot by /u/Hot_Restaurant3243",
    username="Hot_Restaurant3243",
    password="7k%8#248A"
)
try:
    print(f"Authenticated as: {reddit.user.me()}")
except Exception as e:
    print(f"Authentication failed: {e}")

Authenticated as: Hot_Restaurant3243


# Xiaomi 14T Pro

In [11]:

# 设置搜索参数
subreddits = ["Xiaomi", "smartphones", "Android"]  # 要搜索的子版块
search_keywords = ["Xiaomi 14t Pro", "Xiaomi 14T Pro","Xiaomi14T Pro","Xiaomi14t Pro","14T Pro","14t pro","14TPro", "14tpro"]  # 搜索关键词
features = ["camera", "battery", "design", "display", "performance", "price"]  # 特性关键词

# 计算6个月前的时间
start_date = datetime.utcnow() - timedelta(days=180)

def fetch_reddit_posts(subreddit_name, search_terms):
    """
    从指定的subreddit获取相关帖子
    """
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    
    try:
        # 对每个搜索词进行搜索
        for term in search_terms:
            # 使用subreddit的搜索功能
            for post in subreddit.search(term, limit=None):
                post_date = datetime.utcfromtimestamp(post.created_utc)
                
                # 只获取最近6个月的帖子
                if post_date >= start_date:
                    # 检查是否包含任何特性关键词
                    content = (post.title + " " + post.selftext).lower()
                    mentioned_features = [f for f in features if f in content]
                    
                    posts.append({
                        "subreddit": subreddit_name,
                        "title": post.title,
                        "selftext": post.selftext,
                        "created_utc": post_date.strftime('%Y-%m-%d %H:%M:%S'),
                        "score": post.score,
                        "num_comments": post.num_comments,
                        "url": post.url,
                        "features_mentioned": ", ".join(mentioned_features) if mentioned_features else "none",
                        "search_term": term
                    })
                    
    except Exception as e:
        print(f"Error fetching from r/{subreddit_name}: {e}")
    
    return posts

# 获取所有帖子
all_posts = []
for subreddit in subreddits:
    print(f"Fetching from r/{subreddit}...")
    posts = fetch_reddit_posts(subreddit, search_keywords)
    all_posts.extend(posts)
    print(f"Found {len(posts)} posts in r/{subreddit}")

# 转换为DataFrame
df = pd.DataFrame(all_posts)

# 删除重复的帖子
df = df.drop_duplicates(subset=['url'])

# 保存结果
csv_filename = "xiaomi_14Tpro_reddit_posts-180d.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8')
print(f"\nTotal unique posts saved: {len(df)}")
print(f"Data saved to {csv_filename}")

# 显示一些基本统计信息
print("\nPosts by subreddit:")
print(df['subreddit'].value_counts())
print("\nFeatures mentioned:")
print(df['features_mentioned'].value_counts())

Fetching from r/Xiaomi...
Found 814 posts in r/Xiaomi
Fetching from r/smartphones...
Found 328 posts in r/smartphones
Fetching from r/Android...
Found 70 posts in r/Android

Total unique posts saved: 389
Data saved to xiaomi_14Tpro_reddit_posts-180d.csv

Posts by subreddit:
subreddit
Xiaomi         255
smartphones    108
Android         26
Name: count, dtype: int64

Features mentioned:
features_mentioned
none                                            178
camera                                           47
battery                                          41
price                                            33
camera, battery                                  15
camera, battery, price                            9
camera, price                                     7
battery, price                                    7
camera, battery, performance                      7
performance                                       6
camera, performance                               5
display             

# Xiaomi 14 Ultra

In [5]:
from datetime import datetime, timedelta
import pandas as pd
import praw

def fetch_posts_and_comments(reddit):
    """
    获取过去一年内 Xiaomi 14 Ultra 相关帖子及其评论
    """
    # 设置时间范围
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=365)
    
    # 搜索参数
    subreddits = ["Xiaomi", "smartphones", "Android"]
    search_keywords = [
        "Xiaomi 14 Ultra",
        "Xiaomi14Ultra",
        "Xiaomi14 Ultra",
        "14Ultra",
        "14 Ultra"
    ]
    features = ["camera", "battery", "design", "display", "performance", "price"]
    
    posts_data = []
    comments_data = []
    
    for subreddit_name in subreddits:
        print(f"\nSearching in r/{subreddit_name}...")
        subreddit = reddit.subreddit(subreddit_name)
        
        for term in search_keywords:
            print(f"Searching for '{term}'...")
            
            try:
                for post in subreddit.search(term, limit=None):
                    post_date = datetime.utcfromtimestamp(post.created_utc)
                    
                    # 检查帖子是否在时间范围内
                    if start_date <= post_date <= end_date:
                        print(f"Processing post from {post_date.strftime('%Y-%m-%d')}: {post.title[:50]}...")
                        
                        # 检查帖子内容中提到的特性
                        content = (post.title + " " + post.selftext).lower()
                        mentioned_features = [f for f in features if f in content]
                        
                        # 存储帖子信息
                        post_data = {
                            "post_type": "post",
                            "post_id": post.id,
                            "subreddit": subreddit_name,
                            "title": post.title,
                            "selftext": post.selftext,
                            "created_utc": post_date.strftime('%Y-%m-%d %H:%M:%S'),
                            "score": post.score,
                            "num_comments": post.num_comments,
                            "url": post.url,
                            "features_mentioned": ", ".join(mentioned_features) if mentioned_features else "none",
                            "search_term": term,
                            "author": str(post.author),
                            "upvote_ratio": post.upvote_ratio
                        }
                        posts_data.append(post_data)
                        
                        try:
                            # 展开所有评论
                            post.comments.replace_more(limit=None)
                            
                            # 获取所有评论
                            for comment in post.comments.list():
                                comment_date = datetime.utcfromtimestamp(comment.created_utc)
                                # 只保存时间范围内的评论
                                if start_date <= comment_date <= end_date:
                                    comment_data = {
                                        "post_title": post.title,
                                        "post_author": str(post.author),
                                        "post_created_utc": post_date.strftime('%Y-%m-%d %H:%M:%S'),
                                        "post_score": post.score,
                                        "post_url": post.url,
                                        "comment_author": str(comment.author),
                                        "comment_score": comment.score,
                                        "comment_created_utc": comment_date.strftime('%Y-%m-%d %H:%M:%S'),
                                        "comment_body": comment.body
                                    }
                                    comments_data.append(comment_data)
                                
                        except Exception as e:
                            print(f"Error processing comments: {e}")
                            continue
                    
            except Exception as e:
                print(f"Error processing posts: {e}")
                continue
    
    return posts_data, comments_data

def main():
    # 初始化 Reddit API（需要填入你的认证信息）
    
    # 获取帖子和评论数据
    posts_data, comments_data = fetch_posts_and_comments(reddit)
    
    # 转换为DataFrame
    posts_df = pd.DataFrame(posts_data)
    comments_df = pd.DataFrame(comments_data)
    
    # 删除重复的帖子
    posts_df = posts_df.drop_duplicates(subset=['url'], keep='first')
    
    # 生成包含时间范围的文件名
    current_date = datetime.now().strftime('%Y%m%d')
    posts_file = f"xiaomi_14ultra_posts_{current_date}_1year.csv"
    comments_file = f"xiaomi_14ultra_comments_{current_date}_1year.csv"
    
    # 保存结果
    posts_df.to_csv(posts_file, index=False, encoding='utf-8')
    comments_df.to_csv(comments_file, index=False, encoding='utf-8')
    
    # 打印统计信息
    print("\nData Collection Summary:")
    print(f"Time range: {(datetime.utcnow() - timedelta(days=365)).strftime('%Y-%m-%d')} to {datetime.utcnow().strftime('%Y-%m-%d')}")
    print(f"Total posts collected: {len(posts_df)}")
    print(f"Total comments collected: {len(comments_df)}")
    print(f"Average comments per post: {len(comments_df)/len(posts_df):.2f}")
    print(f"\nPosts data saved to: {posts_file}")
    print(f"Comments data saved to: {comments_file}")
    
    # 显示每个subreddit的帖子数量
    print("\nPosts by subreddit:")
    print(posts_df['subreddit'].value_counts())
    
    # 显示特性提及统计
    print("\nFeatures mentioned in posts:")
    features_mentioned = posts_df['features_mentioned'].value_counts()
    print(features_mentioned)
    
    # 显示每月帖子数量统计
    posts_df['month'] = pd.to_datetime(posts_df['created_utc']).dt.to_period('M')
    print("\nPosts by month:")
    print(posts_df['month'].value_counts().sort_index())

if __name__ == "__main__":
    main()


Searching in r/Xiaomi...
Searching for 'Xiaomi 14 Ultra'...
Processing post from 2024-09-13: A week with 14 Ultra : The Report...
Processing post from 2024-05-04: Disappointed of Xiaomi 14 Ultra camera ...
Processing post from 2024-04-05: Xiaomi 14 Ultra BAD NEWS...
Processing post from 2025-01-12: I will be switching to the Xiaomi 15 Ultra from th...
Processing post from 2024-11-21: Xiaomi 15 Ultra in USA, Can I make it work?...
Processing post from 2024-10-28: For 650€, Xiaomi 14 or S23 Ultra or......
Processing post from 2024-02-27: Xiaomi Ultra 14 - 1500€ - Lets talk about it....
Processing post from 2024-03-29: Xiaomi needs to release a statement regarding the ...
Processing post from 2025-02-01: Change from Xiaomi 12 to Xiaomi 14 Ultra...
Processing post from 2024-11-27: S24 Ultra or 14 Ultra...
Processing post from 2025-01-11: Who switched from Samsung S24 Ultra to Xiaomi 14 U...
Processing post from 2025-01-09: Xiaomi 14 ultra vs ?...
Processing post from 2024-04-02: Xiaomi 14